## Exporting a network for production to Onnx

This example is base on the lesson 1, Image classification with Convolutional Neural Networks.
Here, we will see how to train our own image classifier to differentiate tomatoes from potatoes.
We will then export the network to the Onnx format.
Then we will use OpenVino to convert the model and run the inference on the neural compute stick.

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

# Those imports are used for accessing the underlying Pytorch model
from torch.nn import Softmax, Sequential
import torch.onnx
import torchvision
import numpy as np

In [3]:
PATH = "potatoestomatoes/"
sz=224

In [4]:
arch=resnet34
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [5]:
learn.fit(0.01, 3)

epoch      trn_loss   val_loss   accuracy                
    0      0.398667   0.076186   0.979899  
    1      0.219954   0.02086    1.0             
    2      0.141237   0.013367   1.0             



[0.013367459406121517, 1.0]

In [6]:
#test to acess preprocessing parameters
tfms=tfms_from_model(arch, sz)
t = tfms[0]
dir(t)
m,s = t.norm.m, t.norm.s 

In [7]:
log_preds = learn.predict()
print(log_preds.shape)
print(log_preds[:3])
print(log_preds[-3:])
acc = (data.val_y == np.argmax(log_preds, axis=1)).mean() * 100.
print('Accuracy {:0.2f}%'.format(acc))

(199, 2)
[[-0.00072 -7.23136]
 [-0.00008 -9.44336]
 [-0.00237 -6.04511]]
[[-10.75064  -0.00002]
 [ -8.55667  -0.00019]
 [-10.08026  -0.00004]]
Accuracy 100.00%


# Exporting to Onnx
As OpenVino does not support the LogSoftMax layer (See [here](https://software.intel.com/en-us/articles/OpenVINO-Using-ONNX#supported-onnx-layers)), we replace the last layer of the model from a LogSoftMax to a SoftMax.

In [8]:
import copy
# disable precompute to enable the full resnet model + the new fc layers 
learn.precompute = False
# disable training mode, (disabling barch normalisation, dropout, etc)
learn.model.train(False)

dummy_input = Variable(torch.randn(1, 3, sz, sz), requires_grad=True)
model2 = Sequential(copy.deepcopy(learn.model[:-1]), Softmax())
model2 = model2.to('cpu')
model2.train(False)
torch.onnx.export(model2, dummy_input, 'Modelpotatoestomatoes_softmax.onnx')
"Export done"

'Export done'

When going to production, we also need to export the preprocessing done to the images,
here the images are converted to 0-1, resized have the min dimension to be 224, cropped centered,
then we substract the mean and divide by the scale to normilize them.
Finally, we reshape them to be Channel, Width, Height

In [10]:
# Save mean and scale to use in your inference program
trn_tfms, val_tfms = tfms_from_model(arch,sz)
mean, scale = val_tfms.norm.m, val_tfms.norm.s 
mean, scale

(array([0.485, 0.456, 0.406], dtype=float32),
 array([0.229, 0.224, 0.225], dtype=float32))

In [11]:
# Finally, we need to save the class names, as the network will only output the class indicies
data.classes

['potatoes', 'tomatoes']

# Test image preprocessing
Here we run some test to validate that we have the current preprocessing function by comparing our image loading function with the one from fastai/pyrotch.

In [12]:
# get the first 5 validation images
valImgs = data.val_ds[:5][0]
valImgs.shape

(5, 3, 224, 224)

In [13]:
preds = learn.predict_array(valImgs)
acc = (data.val_y[:5] == np.argmax(preds, axis=1)).mean() * 100.
print('learn.predict Accuracy {:0.2f}%'.format(acc))

#second model with SoftMax instead of LogSoftMax
out = model2(torch.from_numpy(valImgs))
softmax = out.detach().numpy()

acc = (data.val_y[:5] == np.argmax(softmax, axis=1)).mean() * 100.
print('softmax Accuracy {:0.2f}%'.format(acc))

learn.predict Accuracy 100.00%
softmax Accuracy 100.00%


In [14]:
data.classes

['potatoes', 'tomatoes']

In [19]:
impaths = [f"{PATH}/valid/potatoes/10. 1557cbd06293972.jpg", f"{PATH}valid/tomatoes/10. fried-green-tomatoes1.jpg"]
import cv2
from matplotlib import pyplot as plt
learn.precompute = False
print('Here, we compare the result of the inference when using our own preprocessing (imt) '
      'with the fastai preprocessing for the original model and the model with the SoftMaxlayer:')
for imp in impaths:
    print(f"Processing image {imp}")
    imfast = open_image(imp)
    # open image to [0-1] RGB, Width Height Channel
    imraw = cv2.imread(imp).astype(np.float32)/255
    imraw = cv2.cvtColor(imraw, cv2.COLOR_BGR2RGB)
    
    # resize and crop center
    r,c, *_ = imraw.shape
    ratio = sz/min(r, c)
    im = cv2.resize(imraw, (max(math.floor(c * ratio), sz), max(math.floor(r * ratio), sz)), interpolation=cv2.INTER_AREA)
    startx = int(np.ceil((im.shape[0] - sz) / 2 ))
    starty = int(np.ceil((im.shape[1] - sz) / 2 ))
    im = im[startx:startx + sz, starty:starty + sz]
    # normalize
    imt = (im - mean) / scale
    imt = imt.transpose((2, 0, 1))
    print("predict tfms", learn.predict_array(val_tfms(imraw)[None]))
    print("predict imt", learn.predict_array(imt[None]))
    print("model2  imt", model2(torch.from_numpy(imt[None])))

    
    #print("imfast", learn.predict_array(val_tfms(imfast)[None]))

Here, we compare the result of the inference when using our own preprocessing (imt) with the fastai preprocessing for the original model and the model with the SoftMaxlayer:
Processing image potatoestomatoes//valid/potatoes/10. 1557cbd06293972.jpg
predict tfms [[-0.00193 -6.25319]]
predict imt [[-0.00193 -6.25319]]
model2  imt tensor([[0.9981, 0.0019]], grad_fn=<SoftmaxBackward>)
Processing image potatoestomatoes/valid/tomatoes/10. fried-green-tomatoes1.jpg
predict tfms [[-6.84312 -0.00107]]
predict imt [[-6.84312 -0.00107]]
model2  imt tensor([[0.0011, 0.9989]], grad_fn=<SoftmaxBackward>)
